### 训练集、测试集、验证集划分，另存为新的csv文件

In [2]:
import os
import pandas as pd
import numpy as np

# ============= 1. 路径设置 =============
root_dir = "/home/charles/HZU/Data_processed/HSML/MCC5/"
csv_path = "/home/charles/HZU/Data_processed/HSML/MCC5/mcc5_speed_10Nm_1000rpm_raw.csv"

print("加载数据：", csv_path)

train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")
test_dir = os.path.join(root_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# ============= 2. 读取数据 =============
df = pd.read_csv(csv_path)
print(f"数据加载成功，共 {len(df)} 条记录")

# 记录原始索引顺序
original_index = df.index.to_numpy()

# ============= 3. 设置划分比例 =============
train_ratio = 0.6
val_ratio = 0.1
test_ratio = 0.3
total = len(df)

# ============= 4. 随机抽取索引，不打乱顺序 =============
rng = np.random.default_rng(42)

# 随机采样，但不改变原始顺序
all_indices = np.arange(total)
rng.shuffle(all_indices)

train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

train_idx = np.sort(all_indices[:train_end])
val_idx = np.sort(all_indices[train_end:val_end])
test_idx = np.sort(all_indices[val_end:])

# ============= 5. 保持原顺序抽取数据 =============
train_df = df.loc[train_idx].copy()
val_df = df.loc[val_idx].copy()
test_df = df.loc[test_idx].copy()

# # ============= 6. 删除最后一列（source_file） =============
# last_col = df.columns[-1]
# print(f"移除最后一列：{last_col}")

# train_df = train_df.iloc[:, :-1]
# val_df = val_df.iloc[:, :-1]
# test_df = test_df.iloc[:, :-1]

# ============= 7. 保存 CSV =============
train_df.to_csv(os.path.join(train_dir, "train.csv"), index=False)
val_df.to_csv(os.path.join(val_dir, "val.csv"), index=False)
test_df.to_csv(os.path.join(test_dir, "test.csv"), index=False)

print("所有数据集已保存（保持原顺序，已去除最后一列）。")

# ============= 8. 保存日志 =============
log_path = os.path.join(root_dir, "split_log.txt")
with open(log_path, "w") as f:
    f.write("KAIST all_class 数据集随机划分日志（保持原顺序+忽略最后一列）\n\n")
    f.write(f"总样本数: {len(df)}\n")
    f.write(f"Train: {len(train_df)}\n")
    f.write(f"Val:   {len(val_df)}\n")
    f.write(f"Test:  {len(test_df)}\n\n")
    f.write("划分比例:\n")
    f.write(f"train_ratio = {train_ratio}\n")
    f.write(f"val_ratio   = {val_ratio}\n")
    f.write(f"test_ratio  = {test_ratio}\n")

print("日志写入完成。")


加载数据： /home/charles/HZU/Data_processed/HSML/MCC5/mcc5_speed_10Nm_1000rpm_raw.csv
数据加载成功，共 1000000 条记录
所有数据集已保存（保持原顺序，已去除最后一列）。
日志写入完成。


### CWT

In [3]:
import os
import pandas as pd
import numpy as np
import pywt

# ======================================================
# 路径
# ======================================================

paths = {
    "train": "/home/charles/HZU/Data_processed/HSML/MCC5/train/train.csv",
    "val":   "/home/charles/HZU/Data_processed/HSML/MCC5/val/val.csv",
    "test":  "/home/charles/HZU/Data_processed/HSML/MCC5/test/test.csv",
}

# ======================================================
# MCC5 通道定义
# ======================================================

VIB_CHANNELS = [
    "motor_vibration_x",
    "motor_vibration_y",
    "motor_vibration_z",
    "gearbox_vibration_x",
    "gearbox_vibration_y",
    "gearbox_vibration_z",
]

COND_CHANNELS = ["speed", "torque"]
LABEL_COL = "class_id"

# ======================================================
# 参数
# ======================================================

WINDOW = 512
STEP   = 256

WIDTHS = np.arange(1, 32)
WAVELET = "morl"

# ======================================================
# Z-score
# ======================================================

def normalize_signal(sig):
    mean = sig.mean(axis=0, keepdims=True)
    std  = sig.std(axis=0, keepdims=True) + 1e-8
    return (sig - mean) / std

# ======================================================
# 处理函数
# ======================================================

def process_csv(split_name, csv_path):

    print(f"\n==================== {split_name.upper()} ====================")
    print("Loading:", csv_path)

    df = pd.read_csv(csv_path)

    X_raw = df[VIB_CHANNELS].values
    speed = df["speed"].values
    torque = df["torque"].values
    y = df[LABEL_COL].values

    N = len(df)
    print("Total samples:", N)

    # -------------------------
    # 振动归一化
    # -------------------------
    X = normalize_signal(X_raw)

    cwt_rows = []
    window_labels = []
    window_speed = []
    window_torque = []
    window_starts = []

    # -------------------------
    # 滑窗 + CWT
    # -------------------------

    for start in range(0, N - WINDOW + 1, STEP):

        chunk = X[start:start+WINDOW]
        chunk_labels = y[start:start+WINDOW]

        # 众数标签
        u, c = np.unique(chunk_labels, return_counts=True)
        win_label = u[np.argmax(c)]

        win_feat = []

        for ch in range(len(VIB_CHANNELS)):

            sig = chunk[:, ch]

            cwt_mat, _ = pywt.cwt(sig, WIDTHS, WAVELET)
            cwt_abs = np.abs(cwt_mat)

            cwt_mean = cwt_abs.mean(axis=1)

            win_feat.extend(cwt_mean.tolist())

        cwt_rows.append(win_feat)
        window_labels.append(win_label)
        window_speed.append(speed[start:start+WINDOW].mean())
        window_torque.append(torque[start:start+WINDOW].mean())
        window_starts.append(start)

    # -------------------------
    # 输出
    # -------------------------

    cwt_rows = np.array(cwt_rows)
    scale_dim = len(WIDTHS)

    print("Windows:", len(cwt_rows))
    print("Feature dim:", cwt_rows.shape[1])

    cols = []
    for ch in range(len(VIB_CHANNELS)):
        for s in range(scale_dim):
            cols.append(f"ch{ch}_scale{s}")

    df_out = pd.DataFrame(cwt_rows, columns=cols)

    df_out.insert(0, "window_start", window_starts)
    df_out.insert(1, "speed", window_speed)
    df_out.insert(2, "torque", window_torque)
    df_out["label"] = window_labels

    save_path = os.path.join(os.path.dirname(csv_path), f"{split_name}_CWT.csv")
    df_out.to_csv(save_path, index=False)

    print("Saved:", save_path)


# ======================================================
# 执行
# ======================================================

for name, path in paths.items():
    process_csv(name, path)

print("\n🎉 MCC5 CWT 处理完成（train / val / test）")



==================== TRAIN ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/train/train.csv
Total samples: 600000
Windows: 2342
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_CWT.csv

==================== VAL ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/val/val.csv
Total samples: 100000
Windows: 389
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_CWT.csv

==================== TEST ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/test/test.csv
Total samples: 300000
Windows: 1170
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_CWT.csv

🎉 MCC5 CWT 处理完成（train / val / test）


### mixed_data->pyg

#### 时间邻点建图

##### CWT数据建图

In [2]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph   # ★ 你要用的函数
import pandas as pd
import os
import torch

# ================== 2️⃣ 根目录 ==================
root_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization"

splits = {
    "train": os.path.join(root_dir, "train/train_CWT.csv"),
    "val":   os.path.join(root_dir, "val/val_CWT.csv"),
    "test":  os.path.join(root_dir, "test/test_CWT.csv"),
}

# ================== 3️⃣ 建图参数可调 ==================
num_edges = 10     # 时间相邻 KNN 数（你自己定）

# ================== 4️⃣ 分别为三个集建图 ==================
for split_name, csv_path in splits.items():

    print(f"\n🚀 开始构建 {split_name} 图结构...")
    assert os.path.exists(csv_path), f"❌ CSV 文件不存在：{csv_path}"

    # === 保存路径：train/pyg/、val/pyg/、test/pyg/ ===
    pyg_dir = os.path.join(os.path.dirname(csv_path), "CWT_pyg")
    os.makedirs(pyg_dir, exist_ok=True)

    print(f"📁 pyg 保存目录: {pyg_dir}")

    # === 调用本地时序建图函数 ===
    nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
        csv_path=csv_path,
        save_dir=pyg_dir,
        num_edges=num_edges
    )

    print(f"   📄 nodes_csv: {nodes_csv}")
    print(f"   📄 edges_csv: {edges_csv}")
    print(f"   📦 graph_pt : {graph_pt}")

    print(f"✅ {split_name} 图构建完成。")

print("\n🎉 三个数据集 (train/val/test) 的图已经全部独立构建完毕！")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib

🚀 开始构建 train 图结构...
📁 pyg 保存目录: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg
✅ 图结构构建完成，共 1639 个节点，16390 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/graph.pt
🧩 特征维度: 124 (已自动忽略首列与标签列)
   📄 nodes_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/nodes.csv
   📄 edges_csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/edges.csv
   📦 graph_pt : /home/charles/HZU

In [ ]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py

import torch
import os
import numpy as np

# ================== 2️⃣ 读取 train 图 ==================
train_graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/KAIST_CWT/normalization/train/CWT_pyg/graph.pt"
assert os.path.exists(train_graph_path), f"❌ 文件不存在：{train_graph_path}"

print(f"📌 加载 train 图：{train_graph_path}")
data = torch.load(train_graph_path)

total_nodes = data.x.size(0)
print(f"节点数 = {total_nodes}")

# ================== 3️⃣ 设置比例 ==================
train_label_ratio = 0.3   # 30% 节点为 train_withlabel

num_withlabel = int(total_nodes * train_label_ratio)
num_nolabel = total_nodes - num_withlabel

print(f"👉 有标签节点：{num_withlabel}  | 无标签节点：{num_nolabel}")

# ================== 4️⃣ 随机生成 mask（保持可复现） ==================
rng = np.random.default_rng(42)

all_indices = np.arange(total_nodes)
rng.shuffle(all_indices)

withlabel_idx = all_indices[:num_withlabel]
nolabel_idx   = all_indices[num_withlabel:]

# 创建 mask
train_withlabel_mask = np.zeros(total_nodes, dtype=bool)
train_nolabel_mask   = np.zeros(total_nodes, dtype=bool)

train_withlabel_mask[withlabel_idx] = True
train_nolabel_mask[nolabel_idx] = True

# ================== 5️⃣ 添加到 Data 对象 ==================
data.train_withlabel_mask = torch.tensor(train_withlabel_mask)
data.train_nolabel_mask   = torch.tensor(train_nolabel_mask)

print("✅ 掩码添加完成：")
print(f"  train_withlabel_mask.sum() = {int(data.train_withlabel_mask.sum())}")
print(f"  train_nolabel_mask.sum()   = {int(data.train_nolabel_mask.sum())}")

# ================== 6️⃣ 保存新的图结构 ==================
save_path = os.path.join(
    os.path.dirname(train_graph_path),
    "train_graph_with_labelmask.pt"
)

torch.save(data, save_path)
print(f"🎉 新图已保存为：{save_path}")



==================== TRAIN ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/train/train.csv
Total samples: 60000
Windows: 233
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/train/train_CWT.csv

==================== VAL ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/val/val.csv
Total samples: 10000
Windows: 38
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/val/val_CWT.csv

==================== TEST ====================
Loading: /home/charles/HZU/Data_processed/HSML/MCC5/test/test.csv
Total samples: 30000
Windows: 116
Feature dim: 186
Saved: /home/charles/HZU/Data_processed/HSML/MCC5/test/test_CWT.csv

🎉 MCC5 CWT 处理完成（train / val / test）
